# Menyiapkan Data
sebelumnya dataset Pemiluu.csv sudah dilakukan preprocessing teks

In [ ]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()

Saving Pemiluu.csv to Pemiluu.csv


In [ ]:
TWEET_DATA = pd.read_csv("Pemiluu.csv", usecols=["label", "tweet_tokens_stemmed"])
TWEET_DATA.columns = ["label", "tweet"]

TWEET_DATA.head()

,label,tweet
0,1,"['akun', 'kloning', 'dukung', 'serang', 'paslo..."
1,2,"['bicara', 'kasihan', 'yaalap', 'air', 'mata',..."
2,3,"['nunggu', 'hasil', 'qc', 'tp', 'nunggu', 'mot..."
3,4,"['kasi', 'oh', 'kasi', 'peluru', 'milyar', 'rw..."
4,5,"['maaf', 'dukung', 'hayo', 'dukung', 'putar']"


In [ ]:
# convert list formated string to list
import ast

def convert_text_list(texts):
    texts = ast.literal_eval(texts)
    return [text for text in texts]

TWEET_DATA["tweet_list"] = TWEET_DATA["tweet"].apply(convert_text_list)


print(TWEET_DATA["tweet_list"][90])

print("\ntype : ", type(TWEET_DATA["tweet_list"][90]))

['tinggal', 'zona', 'aman', 'nyaman', 'maju', 'jakarta', 'hasil', 'sesuai', 'harap', 'ahy']

type :  <class 'list'>


# Menghitung Term Frequency (TF)

In [ ]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

TWEET_DATA["TF_dict"] = TWEET_DATA['tweet_list'].apply(calc_TF)

TWEET_DATA["TF_dict"].head()

0    {'akun': 0.1111111111111111, 'kloning': 0.1111...
1    {'bicara': 0.16666666666666666, 'kasihan': 0.1...
2    {'nunggu': 0.2, 'hasil': 0.1, 'qc': 0.1, 'tp':...
3    {'kasi': 0.2222222222222222, 'oh': 0.111111111...
4    {'maaf': 0.2, 'dukung': 0.4, 'hayo': 0.2, 'put...
Name: TF_dict, dtype: object

In [ ]:
# Check TF result
index = 90

print('%20s' % "term", "\t", "TF\n")
for key in TWEET_DATA["TF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key])

                term 	 TF

             tinggal 	 0.1
                zona 	 0.1
                aman 	 0.1
              nyaman 	 0.1
                maju 	 0.1
             jakarta 	 0.1
               hasil 	 0.1
              sesuai 	 0.1
               harap 	 0.1
                 ahy 	 0.1


# Menghitung Inverse Document Frequency (IDF)

In [ ]:
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(TWEET_DATA["TF_dict"])
DF

{'akun': 6,
 'kloning': 1,
 'dukung': 73,
 'serang': 4,
 'paslon': 20,
 'opini': 1,
 'argumen': 1,
 'pmbenaranjangan': 1,
 'kecoh': 1,
 'bicara': 8,
 'kasihan': 6,
 'yaalap': 1,
 'air': 3,
 'mata': 4,
 'wkwkwkwk': 1,
 'nunggu': 3,
 'hasil': 22,
 'qc': 3,
 'tp': 1,
 'motif': 1,
 'cuit': 4,
 'kayak': 9,
 'pasca': 2,
 'nyungsep': 3,
 'kasi': 3,
 'oh': 2,
 'peluru': 1,
 'milyar': 2,
 'rw': 3,
 'mempan': 1,
 'menangin': 1,
 'pilkada': 84,
 'maaf': 4,
 'hayo': 1,
 'putar': 20,
 'aneh': 2,
 'deh': 7,
 'lebay': 1,
 'sangkut': 1,
 'paut': 1,
 'kandidat': 2,
 'calgub': 1,
 'allah': 12,
 'swt': 3,
 'ucap': 7,
 'syukur': 11,
 'terimakasih': 4,
 'makna': 1,
 'suara': 29,
 'terima': 22,
 'kasih': 19,
 'untuk': 2,
 'kalian': 1,
 'beda': 9,
 'kali': 4,
 'trima': 1,
 'keistiqomahan': 1,
 'rawan': 6,
 'tlh': 1,
 'antar': 1,
 'signifikan': 1,
 'kenang': 2,
 'pidato': 3,
 'kalah': 66,
 'dammnn': 1,
 'politik': 21,
 'cantik': 1,
 'ngorbanin': 1,
 'batal': 3,
 'nyoblos': 7,
 'no': 18,
 'kertas': 1,
 'ngakak

In [ ]:
n_document = len(TWEET_DATA)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict

#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)

In [ ]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
TWEET_DATA["TF-IDF_dict"] = TWEET_DATA["TF_dict"].apply(calc_TF_IDF)

In [ ]:
# Check TF-IDF result
index = 90

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in TWEET_DATA["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", TWEET_DATA["TF_dict"][index][key] ,"\t" , TWEET_DATA["TF-IDF_dict"][index][key])

                term 	         TF 	              TF-IDF

             tinggal 	 0.1 	 0.44998096703302654
                zona 	 0.1 	 0.6109247582764366
                aman 	 0.1 	 0.4722953221644475
              nyaman 	 0.1 	 0.6109247582764366
                maju 	 0.1 	 0.40298060410845293
             jakarta 	 0.1 	 0.23956755160600576
               hasil 	 0.1 	 0.36669005473951616
              sesuai 	 0.1 	 0.48564846142689977
               harap 	 0.1 	 0.380666248977032
                 ahy 	 0.1 	 0.22697952701710547


In [ ]:
# sort descending by value for DF dictionary
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)[:50]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

TWEET_DATA["TF_IDF_Vec"] = TWEET_DATA["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(TWEET_DATA["TF_IDF_Vec"][0])

print("\nmatrix size : ", len(TWEET_DATA["TF_IDF_Vec"][0]))

print first row matrix TF_IDF_Vec Series

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2775921855689045, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4175413695112097, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

matrix size :  50


In [ ]:
# Convert Series to List
TF_IDF_Vec_List = np.array(TWEET_DATA["TF_IDF_Vec"].to_list())

# Sum element vector in axis=0
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))

ranking = pd.DataFrame(data, columns=['term', 'rank'])
ranking.sort_values('rank', ascending=False)

,term,rank
0,menang,32.429365
3,ahok,31.922567
4,jakarta,30.752846
12,hand,30.589765
7,dukung,29.100972
1,ahy,28.231517
5,face,25.959017
8,kalah,25.078346
9,pilih,23.405768
2,pilkada,22.665736
